In [15]:
import torch 
import numpy as np
import unittest


In [16]:
class Generate:
    def generate_bias(out_channels, bias):
        if bias:
            return torch.rand(out_channels)
        else:
            return torch.zeros(out_channels)

    def generate_filter(in_channels, out_channels, kernel_size):
        if type(kernel_size) == tuple:
            return torch.rand(in_channels, out_channels, kernel_size[0], kernel_size[1])
        elif type(kernel_size) == int:
            return torch.rand(in_channels, out_channels, kernel_size, kernel_size)

    def apply_dilation(total_mult, dilation):
        zero_tensor = torch.zeros(
            (total_mult.shape[0] - 1) * dilation + 1,
            (total_mult.shape[1] - 1) * dilation + 1
        )

        for a in range(0, zero_tensor.shape[0], dilation):
            for b in range(0, zero_tensor.shape[1], dilation):
                zero_tensor[a][b] = total_mult[a // dilation][b // dilation]

        return zero_tensor

    def update_feature_map(feature_map, res, i, j, stride, dilation, filter):
        updated_map = np.add(
            feature_map[i * stride:i * stride + (filter.shape[2] - 1) * dilation + 1,
            j * stride:j * stride + (filter.shape[3] - 1) * dilation + 1],
            res
        )
        feature_map[i * stride:i * stride + (filter.shape[2] - 1) * dilation + 1,
        j * stride:j * stride + (filter.shape[3] - 1) * dilation + 1] = updated_map
        return feature_map
    def process_padding(convolution, output_padding, padding):
        for t in range(len(convolution)):
            if output_padding > 0:
                padded = torch.nn.ConstantPad1d((0, output_padding, 0, output_padding), 0)
                convolution[t] = padded(convolution[t])

            convolution[t] = convolution[t][0 + padding:convolution[t].shape[0] - padding,
                             0 + padding:convolution[t].shape[1] - padding]

        return convolution

    def ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros'):
        def convolution(matrix):
            if padding_mode != 'zeros':
                raise Exception('Only "zeros" padding mode is supported in ConvTranspose2d')

            bias_values = generate_bias(out_channels, bias)
            filter = generate_filter(in_channels, out_channels, kernel_size)

            convolution = []

            for l in range(out_channels):
                feature_map = torch.zeros(
                    (matrix.shape[1] - 1) * stride + dilation * (kernel_size - 1) + 1,
                    (matrix.shape[2] - 1) * stride + dilation * (kernel_size - 1) + 1
                )

                for c in range(in_channels):
                    for i in range(0, matrix.shape[1]):
                        for j in range(0, matrix.shape[2]):
                            val = matrix[c][i][j]
                            total_mult = val * filter[c][l]
                            zero_tensor = apply_dilation(total_mult, dilation)
                            feature_map = update_feature_map(feature_map, zero_tensor, i, j, stride, dilation, filter)

                convolution.append(np.add(feature_map, np.full((feature_map.shape), bias_values[l])))

            convolution = process_padding(convolution, output_padding, padding)

            return convolution, filter, torch.tensor(bias_values)

        return convolution
    
class TestGenerate(unittest.TestCase):
    def test_generate_bias(self):
        out_channels = 3
        bias_values = Generate.generate_bias(out_channels, True)
        self.assertEqual(len(bias_values), out_channels)

    def test_generate_filter(self):
        in_channels = 2
        out_channels = 3
        kernel_size = (3, 3)
        filter_tensor = Generate.generate_filter(in_channels, out_channels, kernel_size)
        self.assertEqual(filter_tensor.shape, (in_channels, out_channels, kernel_size[0], kernel_size[1]))
    

if __name__ == '__main__':
    unittest.main()


E
ERROR: C:\Users\arina\AppData\Roaming\jupyter\runtime\kernel-08a7765d-20eb-4d9a-a5ad-0e077de53adb (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\arina\AppData\Roaming\jupyter\runtime\kernel-08a7765d-20eb-4d9a-a5ad-0e077de53adb'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

C:\Users\arina\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3259: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# дополнительное задание

In [13]:
import unittest

class MyFunctions:

    def apply_transpose_padding(matr, transp_stride, pad):
        zero_tensor = np.zeros((((matr.shape[0] - 1) * transp_stride) + 1, ((matr.shape[1] - 1) * transp_stride) + 1))

        for a in range(0, zero_tensor.shape[0], transp_stride):
            for b in range(0, zero_tensor.shape[1], transp_stride):
                zero_tensor[a][b] = matr[a // transp_stride][b // transp_stride]

        pad_matr = np.pad(zero_tensor, pad_width=pad, mode='constant')

        return pad_matr

    def process_output_channel(matrix, filter, bias_values, in_channels, dilation, stride, l):
        feature_map = np.array([])

        for i in range(0, matrix.shape[1] - ((filter.shape[2] - 1) * dilation + 1) + 1, stride):
            for j in range(0, matrix.shape[2] - ((filter.shape[3] - 1) * dilation + 1) + 1, stride):
                total = 0

                for c in range(in_channels):
                    val = matrix[c][i:i + (filter.shape[2] - 1) * dilation + 1:dilation,
                          j:j + (filter.shape[3] - 1) * dilation + 1:dilation]

                    local_sum = (val * filter[l][c]).sum()
                    total = total + local_sum

                feature_map = np.append(feature_map, float(total + bias_values[l]))

        return feature_map.reshape(
            (matrix.shape[1] - ((filter.shape[2] - 1) * dilation + 1)) // stride + 1,
            (matrix.shape[2] - ((filter.shape[3] - 1) * dilation + 1)) // stride + 1)

    def select_padding_mode(padding_mode, padding):
        if padding_mode == 'zeros':
            return torch.nn.ZeroPad2d(padding)
        elif padding_mode == 'reflect':
            return torch.nn.ReflectionPad2d(padding)
        elif padding_mode == 'replicate':
            return torch.nn.ReplicationPad2d(padding)
        elif padding_mode == 'circular':
            pass
        else:
            raise ValueError(f"Unsupported padding mode: {padding_mode}")

    def select_bias(bias, out_channels):
        if bias:
            return torch.rand(out_channels)
        else:
            return torch.zeros(out_channels)

    def myTranspConv2dInverse(in_channels, out_channels, kernel_size, transp_stride=1, padding=0, dilation=1, bias=True, padding_mode='zeros'):
        def convolution(matrix):
            pad = kernel_size - 1
            result_matrix = []

            for matr in matrix:
                pad_matr = apply_transpose_padding(matr, transp_stride, pad)
                result_matrix.append(pad_matr)

            matrix = torch.tensor(result_matrix)
            bias_values = select_bias(bias, out_channels)
            pad = select_padding_mode(padding_mode, padding)
            matrix = pad(matrix)
            filter = np.array(torch.rand(out_channels, in_channels, kernel_size, kernel_size))

            filter_for_transpose = []

            for j in range(out_channels):
                filter_in = []

                for i in range(in_channels):
                    filter_in.append(np.flip(np.array(filter[j][i])))

                filter_for_transpose.append(filter_in)

            filter_for_transpose = torch.tensor(filter_for_transpose)
            filter_for_transpose = filter_for_transpose.reshape(in_channels, out_channels, kernel_size, kernel_size)

            stride = 1
            convolution_result = []

            for l in range(out_channels):
                feature_map = process_output_channel(matrix, filter, bias_values, in_channels, dilation, stride, l)
                convolution_result.append(feature_map)

            return np.array(convolution_result), torch.tensor(np.array(filter_for_transpose)), torch.tensor(np.array(bias_values))

        return convolution

    
class TestMyFunctions(unittest.TestCase):
    def test_apply_transpose_padding(self):
        # Тестирование функции apply_transpose_padding
        matr = np.array([[1, 2], [3, 4]])
        transp_stride = 2
        pad = 1

        padded_matrix = apply_transpose_padding(matr, transp_stride, pad)
        expected_result = np.array([[1, 0, 2, 0],
                                    [0, 0, 0, 0],
                                    [3, 0, 4, 0],
                                    [0, 0, 0, 0]])

        self.assertTrue(np.array_equal(padded_matrix, expected_result))

    def test_process_output_channel(self):
        # Тестирование функции process_output_channel
        matrix = np.random.rand(3, 5, 5)  # Пример входной матрицы
        filter = np.random.rand(2, 3, 3, 3)  # Пример фильтра
        bias_values = np.random.rand(2)  # Пример смещения
        in_channels = 3
        dilation = 1
        stride = 1
        l = 1

        output = process_output_channel(matrix, filter, bias_values, in_channels, dilation, stride, l)
        self.assertEqual(output.shape, (3, 3))  # Проверка размера вывода

if __name__ == '__main__':
    unittest.main()

E
ERROR: C:\Users\arina\AppData\Roaming\jupyter\runtime\kernel-08a7765d-20eb-4d9a-a5ad-0e077de53adb (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\arina\AppData\Roaming\jupyter\runtime\kernel-08a7765d-20eb-4d9a-a5ad-0e077de53adb'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

C:\Users\arina\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3259: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
